**Experiment for obtaining 24 Hr prediction from Dense Model in rainymotion library**

Author: Divya S. Vidyadharan

File use: For predicting 24 Hr precipitation images. 

Date Created: 19-03-21

Last Updated: 20-03-21

Python version: 3.8.2

In [1]:
import h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import sys
import os
module_path = os.path.abspath(os.path.join('/home/divya/divya/OtherNowcastings/rainymotion-master'))
if module_path not in sys.path:
    sys.path.append(module_path)
from rainymotion.models import Dense
from rainymotion.metrics import *

import cv2
import pandas as pd
import wradlib.ipol as ipol # for interpolation
from rainymotion import metrics
from rainymotion import utils
from scipy.ndimage import map_coordinates
import timeit
print(cv2.__version__)
#from tvl1sindysupport import tvl1utilities -in future our own library

ModuleNotFoundError: No module named 'rainymotion'

In [2]:
 times=['0000','0010', '0020', '0030', '0040', '0050',
          '0100', '0110', '0120', '0130', '0140', '0150',
          '0200', '0210', '0220', '0230', '0240', '0250',
          '0300', '0310', '0320', '0330', '0340', '0350',
          '0400', '0410', '0420', '0430', '0440' ,'0450',
          '0500', '0510', '0520', '0530', '0540', '0550',
          
          '0600', '0610', '0620', '0630', '0640', '0650',
          '0700', '0710', '0720', '0730', '0740', '0750',
          '0800', '0810', '0820', '0830', '0840', '0850',
          '0900', '0910', '0920', '0930', '0940', '0950',
          '1000', '1010', '1020', '1030', '1040', '1050',
          '1100', '1110', '1120', '1130', '1140', '1150',
          '1200', '1210', '1220', '1230', '1240', '1250',
          
          
          '1300', '1310', '1320', '1330', '1340', '1350',
          '1400', '1410', '1420', '1430', '1440', '1450',
          '1500', '1510', '1520', '1530', '1540', '1550',
          '1600', '1610', '1620', '1630', '1640', '1650',
          '1700', '1710', '1720', '1730', '1740', '1750',
          '1800', '1810', '1820', '1830', '1840', '1850',
          
          '1900', '1910', '1920', '1930', '1940', '1950',
          '2000', '2010', '2020', '2030', '2040', '2050',
          '2100', '2110', '2120', '2130', '2140', '2150',
          '2200', '2210', '2220', '2230', '2240', '2250',
          '2300', '2310', '2320', '2330', '2340', '2350']


In [3]:
# Common Initialization
eventName = "TyphoonFaxai"
eventDate ="20190908"

#Latitude and Longitude of Typhoon Faxai

lat1 = 32.5
lat2 = 39
long1 = 136
long2 = 143


pred_date = 20190908 #YYYYMMDD
[height, width] = [781,561]
eventNameDate = eventName + "_" + eventDate

startHr = 2
startMin= 40

predStartHr = 300
step = 5 #for rainymotion models


# For radar images
inputFolder = "./ForExperiments/Exp1/RadarImages/HeavyRainfall/For300/"
# outputFolder= "./ForExperiments/Exp1/Results/"
# print(inputFolder)

fileType='.bin'
timeStep = 10 # for Japan Radar Data

modelName = "Dense" 
# startHr = 7# the first hr among for the three input images
# startMin = 30 #
# noOfImages = 3
stepRainyMotion = 5 # 5 minutes
# outputFilePath = outputFolder+modelName+'_'
# outputFilePath = outputFilePath + eventNameDate

# print(outputFilePath)


##recentFramePath##
recentFrameFolder = str(pred_date)+"_set_24Hr_bin" #20190908_set_24Hr_bin
recentFramePath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%recentFrameFolder
print ("\n Recent frame path ",recentFramePath)
inputFolder = recentFramePath
print("\n Input folder is ",inputFolder)
##Output path where predicted images for visual comparison are saved.##
outputimgpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,modelName,"pred_images")
os.makedirs(outputimgpath, exist_ok=True)
print ("\n Output image path is ",outputimgpath)

##Output path where evaluation results are saved as csv files.##
outputevalpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,modelName,"eval_results")
os.makedirs(outputevalpath, exist_ok=True)
print ("\n Output eval results in ",outputevalpath)

savepath = outputimgpath#"Outputs/%i/%s"%(pred_date,pred_times[0])

noOfImages = 3 #  Model needs 24 frames

step = 5
outputFilePath = outputimgpath+'/'
outputFilePath = outputFilePath + eventNameDate

print(outputFilePath)

hrlimit = len(times)
leadsteps = 6
totinputframes = 2


 Recent frame path  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24Hr_bin

 Input folder is  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24Hr_bin

 Output image path is  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Dense/pred_images

 Output eval results in  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Dense/eval_results
/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Dense/pred_images/TyphoonFaxai_20190908


In [4]:
def gettimes24hr(pred_time):
    
   
   # times=np.array(times)
    inptimes = []
    pred_times = []
    index = times.index(pred_time)
    indexlimit = len(times)
    print("Leadsteps are ", leadsteps)
    if (index+leadsteps) < indexlimit:
        pred_times = times[index:index+leadsteps]
    if (index-totinputframes)>=0:
        inptimes = times[index-totinputframes:index]
              
    print("PredTimes:",pred_times)
    print("InpTimes:",inptimes)
    print("Get Time Success..")
    
    return inptimes, pred_times

In [5]:
def readRadarImages(pred_time,inputpath,height,width, noOfImages,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    inputRadarImages = []
    
    i = 0
    index = times.index(pred_time)
#     print(index)
    inputframes = times[index-noOfImages:index]
#     print(len(inputframes))
    while (i<noOfImages):
        
        inputframetime = "_"+inputframes[i]
        i = i +1
        for fileName in files:

            if inputframetime in fileName:
                print("The input image at %s is available",inputframetime)
                print(fileName)
                if fileName.endswith(fileType):

                    
                    inputFileName =recentFramePath+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 
                    inputFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    inputFrame = np.reshape(inputFrame,(height,width))
                    inputFrame = inputFrame.astype('float16') 

                    #print(recentFrame.shape)
                    inputRadarImages.append(inputFrame)
                    #else:
                    #    print("Sorry, unable to find file.")
            
    inputRadarImages = np.stack(inputRadarImages, axis=0)
    print(inputRadarImages.shape)
    return inputRadarImages


**1.2 Dense**

In [6]:
def doDenseNowcasting(startpredtime, saveimages):

    model = Dense()
    model.input_data = readRadarImages(startpredtime,inputFolder,height,width, noOfImages,fileType)
    start = timeit.timeit()
    nowcastDense = model.run()
    end = timeit.timeit()
    sparseTime = end - start
    print("Dense took ",end - start)
    print(nowcastDense.shape)
    for i in range(12):
        outFrameName = outputFilePath + '_'+str(predStartHr+(i*5))+'.png'
    #     print(outFrameName)
        if saveimages:
            matplotlib.image.imsave(outFrameName, nowcastDense[i])
    print("Finished Dense model nowcasting!")
    return nowcastDense

**2. Performance Evaluation**

In [7]:

def getGroundTruthImages(pred_times,leadsteps,recentFramePath,height,width,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    groundTruthImages = []
    
    i = 0
    while (i<leadsteps):
        
        groundtruthtime = "_"+pred_times[i]
        i = i +1
        for fileName in files:

            if groundtruthtime in fileName:
                print("The ground truth at %s is available",groundtruthtime)
                print(fileName)
                if fileName.endswith(fileType):

                    
                    inputFileName =recentFramePath+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 
                    recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    recentFrame = np.reshape(recentFrame,(height,width))
                    recentFrame = recentFrame.astype('float16') 

                    #print(recentFrame.shape)
                    groundTruthImages.append(recentFrame)
                    #else:
                    #    print("Sorry, unable to find file.")
            
    groundTruthImages = np.moveaxis(np.dstack(groundTruthImages), -1, 0)
    #print(groundTruthImages.shape)
    return groundTruthImages

In [8]:
def evaluate(nowcasts):
    fileType = '.bin'
    leadsteps = 6 # 6 for 1 hr prediction, 18 for 3hr prediction
    groundTruthPath = recentFramePath
    print(pred_times)
    groundTruthImgs = getGroundTruthImages(pred_times,leadsteps,groundTruthPath,height,width,fileType)
  
    maelist = []
    farlist = []
    podlist= []
    csilist= []
    thres =1.0
    
    noOfPrecipitationImages = 6
    j = 0  # using another index to skip 5min interval data from rainymotion
    for i in range(noOfPrecipitationImages):
        mae = MAE(groundTruthImgs[i],nowcasts[j])
        far = FAR(groundTruthImgs[i],nowcasts[j], threshold=0.1)
        pod = POD(groundTruthImgs[i],nowcasts[j], threshold=0.1)
        csi = CSI(groundTruthImgs[i],nowcasts[j],thres)
        maelist.append(mae)
        farlist.append(far)
        podlist.append(pod)
        csilist.append(csi)
        j = j + 2
    return csilist,maelist,farlist,podlist


    
    

**2. 24 Hr Prediction**

In [9]:
startpredtime = '0110' #'1100'

index = times.index(startpredtime)
indexlimit = times.index('2250') # Since we have only 6 more ground truths available from this time
print(index)
print("Last prediction is at index ", indexlimit)
csilist = []
maelist = []
podlist = []
farlist = []
pred_time = startpredtime
while index<8:#indexlimit:#len(times):
    print(times[index])
    saveimages = 0
    if (index==66):
        saveimages=1
    intimes, pred_times = gettimes24hr(pred_time)
    nowcasts = doDenseNowcasting(pred_time,saveimages)
    
    csi,mae,far,pod = evaluate(nowcasts)
    
    csilist.append(csi)
    maelist.append(mae)
    podlist.append(pod)
    farlist.append(far)
    
    index = index+1
    pred_time = times[index] 
        

7
Last prediction is at index  137
0110
Leadsteps are  6
PredTimes: ['0110', '0120', '0130', '0140', '0150', '0200']
InpTimes: ['0050', '0100']
Get Time Success..
The input image at %s is available _0040
20190908_004000.000.bin
The input image at %s is available _0050
20190908_005000.000.bin
The input image at %s is available _0100
20190908_010000.000.bin
(3, 781, 561)


AttributeError: module 'cv2.optflow' has no attribute 'createOptFlow_DIS'

In [40]:
DISOpticalFlow_create
# For debugging
print(len(maelist))
print("\n\n")
print(len(csilist))
print("\n\n")
print(len(podlist))
print("\n\n")
print(len(farlist))


78



78



78



78


**To save results in excel workbook**

In [41]:
import xlwt 
from xlwt import Workbook 
  
# Workbook is created 
wb = Workbook() 


In [42]:
def writeinexcelsheet(sheetname, wb, results):
     
    sheet1 = wb.add_sheet(sheetname) 
    sheet1.write(0, 0, 'Pred.no.') 
    sheet1.write(0, 1, 't (pred start time)') 
    sheet1.write(0, 2, 't + 10') 
    sheet1.write(0, 3, 't + 20') 
    sheet1.write(0, 4, 't + 30') 
    sheet1.write(0, 5, 't + 40') 
    sheet1.write(0, 6, 't + 50') 
    
    
    col = 0

    rows = len(results)
    cols = len(results[0])
    print(cols)
    for rowno in range(rows):
        
        sheet1.write(rowno+1,0,rowno+1)   
        for col in range(cols):
#             print(rowno+1,col+1,results[rowno][col])
            sheet1.write(rowno+1,col+1,results[rowno][col].astype('float64'))
               
#          sheet1.write(row, col, str(data))
#          print(row,col,data)
    
    
    

In [46]:
writeinexcelsheet('CSI',wb,csilist)
writeinexcelsheet('MAE',wb,maelist)
writeinexcelsheet('FAR',wb,farlist)
writeinexcelsheet('POD',wb,podlist)
excelpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/Dense/eval_results/"
excelpath = excelpath + 'resultsDense.xls'
wb.save(excelpath)


6
6
6
6
